In [1]:
import pandas as pd
import os
from tqdm.notebook import tqdm
import shutil


In [2]:
DATA_ROOT = '/raid/siim-covid19-detection/'

In [3]:
train = pd.read_csv(f'{DATA_ROOT}train_image_level_folded_v3.csv')
train

,id,boxes,label,StudyInstanceUID,series_id,height,width,fold,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,81456c9c5423,3488,4256,3,0,1,0,0
1,000c3a3f293f_image,NaN,none 1 0 0 1 1,ff0879eb20ed,d8a644cc4f93,2320,2832,3,1,0,0,0
2,0012ff7358bc_image,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,22897cd1daa0,2544,3056,1,0,1,0,0
3,001398f4ff4f_image,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,4d47bc042ee6,3520,4280,1,0,0,0,1
4,001bd15d1891_image,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,49170afa4f27,2800,3408,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6112,ffcc6edd9445_image,NaN,none 1 0 0 1 1,7e6c68462e06,98011bca5fd2,4240,3480,2,1,0,0,0
6113,ffd91a2c4ca0_image,NaN,none 1 0 0 1 1,8332bdaddb6e,f346cda52da0,2800,3408,0,1,0,0,0
6114,ffd9b6cf2961_image,"[{'x': 2197.38566, 'y': 841.07361, 'width': 31...",opacity 1 2197.38566 841.07361 2513.80265 1292...,7eed9af03814,668a64e3a5fa,2388,3050,1,0,1,0,0
6115,ffdc682f7680_image,"[{'x': 2729.27083, 'y': 332.26044, 'width': 14...",opacity 1 2729.27083 332.26044 4225.52099 2936...,a0cb0b96fb3d,ccf363aa080f,3488,4256,3,0,1,0,0


In [4]:
os.makedirs(f'{DATA_ROOT}yolo_txt/',exist_ok=True)

In [5]:
for i in range(train.shape[0]):
    a = train.loc[i,'id'].replace("_image", "")
    f = open(f'{DATA_ROOT}yolo_txt/{a}.txt', 'w')
    b = train.loc[i, 'label'].split()
    b_len = int(len(b) / 6)
    dim1 = train.loc[i,'width']  # ?
    dim0 = train.loc[i,'height']  # ?
    if b[0] == 'none':
        class_id = 1
        b = ['none',1,0,0,dim1,dim0]
    else:
        class_id = 0
    

    for j in range(b_len):
        x_mid = str((float(b[6*j+2]) + float(b[6*j+4])) / 2 / dim1)
        y_mid = str((float(b[6*j+3]) + float(b[6*j+5])) / 2 / dim0)
        w = str((float(b[6*j+4]) - float(b[6*j+2])) / dim1)
        h = str((float(b[6*j+5]) - float(b[6*j+3])) / dim0)
        f.write(f'{class_id}'+' '+x_mid +' '+y_mid+' '+w+' '+h+' ')
        f.write('\n')

    f.close()

In [6]:
classes = ['0. opacity', '1. none']
train[f'image_path'] = f'{DATA_ROOT}train_05/' + train.StudyInstanceUID.astype(str) + "/" + train.series_id.astype(str) + "/" + train["id"].str.replace("_image", "") + ".png"
train.head()

,id,boxes,label,StudyInstanceUID,series_id,height,width,fold,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,image_path
0,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,81456c9c5423,3488,4256,3,0,1,0,0,/raid/siim-covid19-detection/train_05/5776db0c...
1,000c3a3f293f_image,NaN,none 1 0 0 1 1,ff0879eb20ed,d8a644cc4f93,2320,2832,3,1,0,0,0,/raid/siim-covid19-detection/train_05/ff0879eb...
2,0012ff7358bc_image,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,22897cd1daa0,2544,3056,1,0,1,0,0,/raid/siim-covid19-detection/train_05/9d514ce4...
3,001398f4ff4f_image,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,4d47bc042ee6,3520,4280,1,0,0,0,1,/raid/siim-covid19-detection/train_05/28dddc85...
4,001bd15d1891_image,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,49170afa4f27,2800,3408,1,0,1,0,0,/raid/siim-covid19-detection/train_05/dfd9fdd8...


In [7]:
train_files = []
val_files   = []
for fold in range(5):

    val_files += [list(train[train.fold==fold].image_path.unique())]
    train_files += [list(train[train.fold!=fold].image_path.unique())]

    
    print(len(train_files[fold]), len(val_files[fold]))

4896 1221
4886 1231
4892 1225
4897 1220
4897 1220


In [8]:
for fold in range(5):
    os.makedirs(f'{DATA_ROOT}yolo_fold{fold}/labels/train', exist_ok = True)
    os.makedirs(f'{DATA_ROOT}yolo_fold{fold}/labels/val', exist_ok = True)
    os.makedirs(f'{DATA_ROOT}yolo_fold{fold}/images/train', exist_ok = True)
    os.makedirs(f'{DATA_ROOT}yolo_fold{fold}/images/val', exist_ok = True)
    label_dir = f'{DATA_ROOT}yolo_txt/'
    for file in tqdm(train_files[fold]):
        shutil.copy(file, f'{DATA_ROOT}yolo_fold{fold}/images/train')
        filename = file.split('/')[-1].split('.')[0]
        shutil.copy(os.path.join(label_dir, filename+'.txt'), f'{DATA_ROOT}yolo_fold{fold}/labels/train')

    for file in tqdm(val_files[fold]):
        shutil.copy(file, f'{DATA_ROOT}yolo_fold{fold}/images/val')
        filename = file.split('/')[-1].split('.')[0]
        shutil.copy(os.path.join(label_dir, filename+'.txt'), f'{DATA_ROOT}yolo_fold{fold}/labels/val')